# Gaussian Naive Bayes Classifier

Naive bayes classifier uses baye's theorem to classify samples with strong (naïve) independence assumptions between the features. Gaussian naive bayes is an extension of naive bayes classifier, which deals with points modelled with guassian distribution.

The points of the test set are classified using the naive Bayes classifier, where for a given x, p(x|ωi) is estimated as

>$
p(x \mid \omega_i)=\Pi_{j=1}^f \frac{1}{\sqrt{2\pi\sigma^2_{ij}}}\,e^{ -\frac{(x(j)-m_{ij})^2}{2\sigma^2_{ij}} }, i=1,2,...c
$

where x( j) is the jth component of x, f is the no. of features, and c is the no. of classes.


In [1]:
"""
Gaussian naive bayes classifier

@Author: Ajay Biswas
National Institute of Technology, Rourkela
"""

import numpy as np
from sklearn.naive_bayes import GaussianNB
from collections import Counter  
import math

def misclassifications(X,Y):
  correct_count = 0
  for i in range(len(X)):
    if(X[i] == Y[i]):
      correct_count = correct_count + 1
  return len(X) - correct_count

def MER_Error(X,Y):
  correct_count = 0
  for i in range(len(X)):
    if(X[i] == Y[i]):
      correct_count = correct_count + 1
  MER_val = 1 - (correct_count/len(X))
  return MER_val

def estimator(X,V,Y):  
   means = X
   variances = V
   no_features = len(means)
   p = 1
   for i in range(no_features):
       exponent =  math.exp(-((Y[i] - means[i]) ** 2 / (2 * variances[i])))
       fraction = (1 / (math.sqrt(2 * math.pi * variances[i])))
       p = p * exponent*fraction     
   return p
       

def naive_bayes_train(X,y):
    # X contains n dimensional features
    # y contains true label
    # find mean of X
    
    no_of_samples,no_of_features = X.shape
        
    unique_classes = set(y)
    sc = dict(Counter(y))
    frequency_per_class = [sc[i] for i in unique_classes]
    
    # group samples classwise and find their centroid
    start = 0
    means = []
    variances = []
    for value in frequency_per_class:
        tempList = X[start:start+value,:]
        each_mean = tempList.mean(0)
        each_variance = tempList.var(0)
        means.append(each_mean)
        variances.append(each_variance)
        start = value
        
    return np.array(means),np.array(variances)

def naive_bayes_test(X,means,variances,y):
    # X contains n dimensional features
    # y contains true class labels
    # model is array of centroids for each class present in y respectively
    
    num_rows, num_cols = means.shape
    X_rows, X_cols = X.shape
    unique_classes = set(y)
    y_len = len(y)
    
    # no. of labels must match no. of rows in the model (array of centroids)
    if(len(unique_classes)!= num_rows):
        return None
    
    # no. of features must match with the no. of dimensions of the model
    if(X_cols != num_cols):
        return None
    
    # predicted labels
    predicted = [0]*y_len
   
    # test each point against each class and assign label based on max probability
    prob = 0
    max_prob = 0
    i = 0
    j = 0
    for test_point in X:
        j = 0
        for each_mean,each_variance in means,variances:
            prob = estimator(each_mean,each_variance,test_point)
            
            if(max_prob < prob):
                max_prob = prob
                label = j
            j+=1
          
        predicted[i] = label    
        max_prob = 0
        i+=1
        
    return np.array(predicted) 


def main():
    
    # Covariance Matrix of two classes
    S1 =np.array([[0.8, 0.2, 0.1, 0.05, 0.01],
        [0.2, 0.7, 0.1, 0.03, 0.02],
        [0.1, 0.1, 0.8, 0.02, 0.01],
        [0.05, 0.03, 0.02, 0.9, 0.01],
        [0.01, 0.02, 0.01, 0.01, 0.8]])
    
    S2 =np.array([[0.9, 0.1, 0.05, 0.02, 0.01],
        [0.1, 0.8, 0.1, 0.02, 0.02],
        [0.05, 0.1, 0.7, 0.02, 0.01],
        [0.02, 0.02, 0.02, 0.6, 0.02],
        [0.01, 0.02, 0.01, 0.02, 0.7]])
    
    # Mean of two classes
    mean = [0,0,0,0,0]
    mean2 = [1,1,1,1,1]
    
    # training
    X_h1 = np.random.multivariate_normal(mean, S1, 25)
    X_h2 = np.random.multivariate_normal(mean2, S2, 25)
    X = np.concatenate((X_h1,X_h2))
    y = np.concatenate(([0]*25,[1]*25))    
    means,variances = naive_bayes_train(X,y)

    print('\nEstimated means of the two classes:\n',means)
    print('\nEstimated variances of the two classes:\n',variances)
    
    # testing
    X2_h1 = np.random.multivariate_normal(mean, S1, 5000)
    X2_h2 = np.random.multivariate_normal(mean2, S2, 5000)
    X2 = np.concatenate((X2_h1,X2_h2))  
    y2 = np.concatenate(([0]*5000,[1]*5000))    
    L = naive_bayes_test(X2,means,variances,y2)  
    
    error_rate = MER_Error(y2, L)
    misclassification =  misclassifications(y2, L)

    print('\nNo. of Misclassifications: ',misclassification)
    print('\nError Probability: ',error_rate)

if __name__=="__main__": 
    main() 
    



Estimated means of the two classes:
 [[-0.02216921 -0.24041423 -0.25386841 -0.3253863   0.19904747]
 [ 1.14836461  1.06565619  1.14682332  0.99270391  0.82954387]]

Estimated variances of the two classes:
 [[0.95517139 0.88972547 0.47134716 0.85788958 0.76288744]
 [0.89695068 1.17387662 0.5825118  0.49658455 0.3338967 ]]

No. of Misclassifications:  1560

Error Probability:  0.15600000000000003
